# **PROJET FINE-TUNING LLM**

## ***Importer les librairies usuelles*** ##

In [1]:
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install trl
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.8 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hz5r1ohj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hz5r1ohj
  Resolved https://github.com/huggingface/transformers.git to commit edbabf6b82b7cbe73965d75324bec2b6c16c1008
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10500412 sha256=aeceb8ecb3b054db02030921562dc56ea86c1713da15c8ae529b0c99c74bc460
  Stored in directory: /tmp/pip-ephem-wheel-cache-_q1a3b3s/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:


In [2]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset
import random
import torch
import pandas as pd
from numba import cuda
from transformers import GPT2LMHeadModel, GPTNeoForCausalLM, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge_score import rouge_scorer
import re

## ***Importer le datset*** ##

In [3]:
dataset = load_dataset("manu/french_poetry")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/486 [00:00<?, ?B/s]

(…)-00000-of-00001-927b7145806656e5.parquet:   0%|          | 0.00/2.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
# Extraire le Datset grâce à la clé "train"
df = dataset['train'].to_pandas()
df.head()

,title,poet,text,link,id
0,Les deux bonnes soeurs,Charles Baudelaire (1821-1867),Poésie : Les deux bonnes soeurs\nTitre : Les d...,https://www.poesie-francaise.fr/charles-baudel...,charles-baudelaire-poeme-les-deux-bonnes-soeurs
1,"Ami, j'ai quitté vos fêtes",Victor Hugo (1802-1885),"Poésie : Ami, j'ai quitté vos fêtes\nTitre : A...",https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-ami-j-ai-quitte-vos-fetes
2,Je ne vois pas pourquoi je ferais autre chose,Victor Hugo (1802-1885),Poésie : Je ne vois pas pourquoi je ferais aut...,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-je-ne-vois-pas-pourquoi-je-f...
3,À Mlle Fanny de P,Victor Hugo (1802-1885),Poésie : À Mlle Fanny de P\nTitre : À Mlle Fan...,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-a-Mlle-Fanny-de-P
4,L'angélus du matin,Paul Verlaine (1844-1896),Poésie : L'angélus du matin\nTitre : L'angélus...,https://www.poesie-francaise.fr/paul-verlaine/...,paul-verlaine-poeme-langelus-du-matin


## ***Explorer le Dataset***

In [5]:
df.dtypes

,0
title,object
poet,object
text,object
link,object
id,object


In [6]:
df.shape

(1821, 5)

In [7]:
df['text'].iloc[50]

"Poésie : Un dahlia\nTitre : Un dahlia\nPoète : Paul Verlaine (1844-1896)\nRecueil : Poèmes saturniens (1866).\nCourtisane au sein dur, à l'oeil opaque et brun\nS'ouvrant avec lenteur comme celui d'un boeuf,\nTon grand torse reluit ainsi qu'un marbre neuf.\nFleur grasse et riche, autour de toi ne flotte aucun\nArôme, et la beauté sereine de ton corps\nDéroule, mate, ses impeccables accords.\nTu ne sens même pas la chair, ce goût qu'au moins\nExhalent celles-là qui vont fanant les foins,\nEt tu trônes, Idole insensible à l'encens.\n— Ainsi le Dahlia, roi vêtu de splendeur,\nElève sans orgueil sa tête sans odeur,\nIrritant au milieu des jasmins agaçants !"

## ***Effectuer le preprocessing sur le dataset*** ##

In [8]:
# Récupérer la section poème positionnée à l'indice 4
df['text'] = df['text'].str.split(':').str[4]

# Garder la section poème positionnée après le premier "\n"
df['text'] = df['text'].str.split('\n', n=1).str[1]

# Supprimer l'intégralité des NAN (poèmes structure non conforme)
#df['text'] = df['text'].dropna()

In [9]:
df.head(15)

,title,poet,text,link,id
0,Les deux bonnes soeurs,Charles Baudelaire (1821-1867),Sonnet.\nLa Débauche et la Mort sont deux aima...,https://www.poesie-francaise.fr/charles-baudel...,charles-baudelaire-poeme-les-deux-bonnes-soeurs
1,"Ami, j'ai quitté vos fêtes",Victor Hugo (1802-1885),NaN,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-ami-j-ai-quitte-vos-fetes
2,Je ne vois pas pourquoi je ferais autre chose,Victor Hugo (1802-1885),Je ne vois pas pourquoi je ferais autre chose\...,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-je-ne-vois-pas-pourquoi-je-f...
3,À Mlle Fanny de P,Victor Hugo (1802-1885),NaN,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-a-Mlle-Fanny-de-P
4,L'angélus du matin,Paul Verlaine (1844-1896),NaN,https://www.poesie-francaise.fr/paul-verlaine/...,paul-verlaine-poeme-langelus-du-matin
5,J'ai tout donné pour rien,Théophile Gautier (1811-1872),"On peut m'aimer sans honte,\nLa couronne de co...",https://www.poesie-francaise.fr/theophile-gaut...,theophile-gautier-poeme-jai-tout-donne-pour-rien
6,La dure épreuve va finir,Paul Verlaine (1844-1896),La dure épreuve va finir,https://www.poesie-francaise.fr/paul-verlaine/...,paul-verlaine-poeme-la-dure-epreuve-va-finir
7,À deux beaux yeux,Théophile Gautier (1811-1872),Vous avez un regard singulier et charmant ;\nC...,https://www.poesie-francaise.fr/theophile-gaut...,theophile-gautier-poeme-a-deux-beaux-yeux
8,Écrit en 1827,Victor Hugo (1802-1885),NaN,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-ecrit-en-1827
9,Lorsque l'enfant paraît,Victor Hugo (1802-1885),NaN,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-lorsque-l-enfant-parait


In [10]:
df['text'][0]

"Sonnet.\nLa Débauche et la Mort sont deux aimables filles,\nProdigues de baisers et riches de santé,\nDont le flanc toujours vierge et drapé de guenilles\nSous l'éternel labeur n'a jamais enfanté.\nAu poète sinistre, ennemi des familles,\nFavori de l'enfer, courtisan mal tenté,\nTombeaux et lupanars montrent sous leurs charmilles\nUn lit que le remords n'a jamais fréquenté.\nEt la bière et l'alcôve en blasphèmes fécondes\nNous offrent tour à tour, comme deux bonnes soeurs,\nDe terribles plaisirs et d'affreuses douceurs.\nQuand veux-tu m'enterrer, Débauche aux bras immondes ?\nÔ Mort, quand viendras-tu, sa rivale en attraits,\nSur ses myrtes infects enter tes noirs cyprès ?"

In [11]:
df.shape

(1821, 5)

## ***Suppression des valeurs NaN aprés transformation ne respectant pas la structure globale***

In [12]:
# Détecter les valeurs NaN
missing_values = df.text.isnull().sum()
print(missing_values)

412


In [13]:
# Supprimer les valeurs Nan
df = df.dropna(subset=['text']).reset_index(drop =True)

In [14]:
df.shape

(1409, 5)

In [15]:
# Compter les valeurs NaN après suppression
missing_values = df.text.isnull().sum()
print(missing_values)

0


In [16]:
df['title'].iloc[4]


'À deux beaux yeux'

In [17]:
df['text'].iloc[4]

"Vous avez un regard singulier et charmant ;\nComme la lune au fond du lac qui la reflète,\nVotre prunelle, où brille une humide paillette,\nAu coin de vos doux yeux roule languissamment ;\nIls semblent avoir pris ses feux au diamant ;\nIls sont de plus belle eau qu'une perle parfaite,\nEt vos grands cils émus, de leur aile inquiète,\nNe voilent qu'à demi leur vif rayonnement.\nMille petits amours, à leur miroir de flamme,\nSe viennent regarder et s'y trouvent plus beaux,\nEt les désirs y vont rallumer leurs flambeaux.\nIls sont si transparents, qu'ils laissent voir votre âme,\nComme une fleur céleste au calice idéal\nQue l'on apercevrait à travers un cristal."

In [18]:
# Suppression des colonnes inutiles
df = df.drop(columns = ['poet', 'link', 'id'])

In [19]:
df.shape

(1409, 2)

In [20]:
df.head()

,title,text
0,Les deux bonnes soeurs,Sonnet.\nLa Débauche et la Mort sont deux aima...
1,Je ne vois pas pourquoi je ferais autre chose,Je ne vois pas pourquoi je ferais autre chose\...
2,J'ai tout donné pour rien,"On peut m'aimer sans honte,\nLa couronne de co..."
3,La dure épreuve va finir,La dure épreuve va finir
4,À deux beaux yeux,Vous avez un regard singulier et charmant ;\nC...


In [21]:
df.text[5]

"Ah ! Si vous saviez comme on pleure\nDe vivre seul et sans foyers,\nQuelquefois devant ma demeure\nVous passeriez.\nSi vous saviez ce que fait naître\nDans l'âme triste un pur regard,\nVous regarderiez ma fenêtre\nComme au hasard.\nSi vous saviez quel baume apporte\nAu cœur la présence d'un cœur,\nVous vous assoiriez sous ma porte\nComme une sœur.\nSi vous saviez que je vous aime,\nSurtout si vous saviez comment,\nVous entreriez peut-être même\nTout simplement."

In [22]:
# Convertir le dataset en CSV
df.to_csv('poems_dataset.csv', index=False, encoding='utf-8')

In [23]:
#df = pd.read_csv('poems_prompts_dataset.csv')

In [24]:
"""from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')
generator("EleutherAI has", do_sample=True, min_length=50)"""

'from transformers import pipeline\ngenerator = pipeline(\'text-generation\', model=\'gpt2\')\ngenerator("EleutherAI has", do_sample=True, min_length=50)'

In [25]:
"""prompt = "Génère un poème sur l'amitié"
res = generator(prompt, max_length = 200, top_k = 80, top_p = 0.90, do_sample = True, temperature = 1.1)

print(res[0]['generated_text'])"""

'prompt = "Génère un poème sur l\'amitié"\nres = generator(prompt, max_length = 200, top_k = 80, top_p = 0.90, do_sample = True, temperature = 1.1)\n\nprint(res[0][\'generated_text\'])'

In [26]:
#generator("Génère un poème sur le thème du courage", do_sample = True, max_length = 200, min_length = 150, temperature = 1, top_k = 80)

In [27]:
df.shape

(1409, 2)

In [28]:
df.text[0]

"Sonnet.\nLa Débauche et la Mort sont deux aimables filles,\nProdigues de baisers et riches de santé,\nDont le flanc toujours vierge et drapé de guenilles\nSous l'éternel labeur n'a jamais enfanté.\nAu poète sinistre, ennemi des familles,\nFavori de l'enfer, courtisan mal tenté,\nTombeaux et lupanars montrent sous leurs charmilles\nUn lit que le remords n'a jamais fréquenté.\nEt la bière et l'alcôve en blasphèmes fécondes\nNous offrent tour à tour, comme deux bonnes soeurs,\nDe terribles plaisirs et d'affreuses douceurs.\nQuand veux-tu m'enterrer, Débauche aux bras immondes ?\nÔ Mort, quand viendras-tu, sa rivale en attraits,\nSur ses myrtes infects enter tes noirs cyprès ?"

## ***Begining of NLP*** ##

#***Imports libs here***#

In [29]:
#from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, default_data_collator
from transformers import DataCollatorForLanguageModeling, default_data_collator
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer

In [30]:
# Apprentissage masqué (Masked Language Modeling, MLM) pour des modèles comme BERT.
# mlm : Un booléen qui indique si le masquage pour MLM doit être appliqué (par défaut, True).

"""def load_data_collator(tokenizer, mlm = False):
  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=mlm, )
  return data_collator"""

'def load_data_collator(tokenizer, mlm = False):\n  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=mlm, )\n  return data_collator'

In [31]:
df['combined'] = '[Q]' + df['title'] + ' ' + '[A]' + df['text']

In [32]:
df.head()

,title,text,combined
0,Les deux bonnes soeurs,Sonnet.\nLa Débauche et la Mort sont deux aima...,[Q]Les deux bonnes soeurs [A]Sonnet.\nLa Débau...
1,Je ne vois pas pourquoi je ferais autre chose,Je ne vois pas pourquoi je ferais autre chose\...,[Q]Je ne vois pas pourquoi je ferais autre cho...
2,J'ai tout donné pour rien,"On peut m'aimer sans honte,\nLa couronne de co...",[Q]J'ai tout donné pour rien [A]On peut m'aime...
3,La dure épreuve va finir,La dure épreuve va finir,[Q]La dure épreuve va finir [A]La dure épreuve...
4,À deux beaux yeux,Vous avez un regard singulier et charmant ;\nC...,[Q]À deux beaux yeux [A]Vous avez un regard si...


In [33]:
from sklearn.model_selection import train_test_split
# Fractionner en train/test
train_data, test_data = train_test_split(df['combined'], test_size=0.2, random_state=42)

In [34]:
# Convertir le dataset en datframe hugging face
from datasets import Dataset, DatasetDict

In [35]:
# Convertir en Dataset Hugging Face
train_dataset = Dataset.from_dict({"text": train_data})
test_dataset = Dataset.from_dict({"text": test_data})
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

In [36]:
#from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Afficher les token par défaut
# Afficher les tokens spéciaux par défaut
print("Tokens spéciaux par défaut :", tokenizer.special_tokens_map)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Tokens spéciaux par défaut : {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}


In [37]:
#from transformers import AutoModelForCausalLM
# Ajouter un nouveau token spécial
#tokenizer.add_special_tokens({"unk_token": "<|unknown|>"})
#tokenizer.add_special_tokens({"pad_token": "<|unknown|>"})

# Charger le modèle
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [38]:
print("The max model length is {} for this model".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))

The max model length is 1024 for this model
The beginning of sequence token <|endoftext|> token has the id 50256
The end of sequence token <|endoftext|> has the id 50256


In [39]:
tokenizer.vocab_size

50257

In [40]:
tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [41]:
#tokenizer.all_special_tokens

In [42]:
#tokenizer.eos_token_id

In [43]:
#tokenizer.max_model_input_sizes

In [44]:
"""sentence = 'I am a Data Science and Artificial Inteligence Dev who know to train a model'
input_ids  = tokenizer.encode(sentence, return_tensors = 'pt')"""

"sentence = 'I am a Data Science and Artificial Inteligence Dev who know to train a model'\ninput_ids  = tokenizer.encode(sentence, return_tensors = 'pt')"

In [45]:
#input_ids

In [46]:
#tokenizer.decode(input_ids[0][7])

In [47]:
print("Taille du vocabulaire du tokenizer :", len(tokenizer))
print("Taille du vocabulaire du modèle :", model.config.vocab_size)

Taille du vocabulaire du tokenizer : 50257
Taille du vocabulaire du modèle : 50257


In [48]:
output_dir = './results'
tokenizer.save_pretrained(output_dir)
model.save_pretrained(output_dir)

## ***Ajouter les tokens préalablement créés au tokenizer***

In [49]:
df.head()

,title,text,combined
0,Les deux bonnes soeurs,Sonnet.\nLa Débauche et la Mort sont deux aima...,[Q]Les deux bonnes soeurs [A]Sonnet.\nLa Débau...
1,Je ne vois pas pourquoi je ferais autre chose,Je ne vois pas pourquoi je ferais autre chose\...,[Q]Je ne vois pas pourquoi je ferais autre cho...
2,J'ai tout donné pour rien,"On peut m'aimer sans honte,\nLa couronne de co...",[Q]J'ai tout donné pour rien [A]On peut m'aime...
3,La dure épreuve va finir,La dure épreuve va finir,[Q]La dure épreuve va finir [A]La dure épreuve...
4,À deux beaux yeux,Vous avez un regard singulier et charmant ;\nC...,[Q]À deux beaux yeux [A]Vous avez un regard si...


In [50]:
print(df.columns)

Index(['title', 'text', 'combined'], dtype='object')


In [51]:
#df['combined'] = '[Q]' + df['prompt'] + ' ' + '[A]' + df['text']

In [52]:
#df['combined'] = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

In [53]:
train_dataset[0]

{'text': '[Q]Sapho [A]Fatal rocher, profond abîme !\nJe vous aborde sans effroi !\nVous allez à Vénus dérober sa victime '}

In [54]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 1127
})

In [55]:
test_dataset

Dataset({
    features: ['text'],
    num_rows: 282
})

In [56]:
# Vérifier et définir un token de padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Utiliser eos_token comme pad_token
    # Ou ajouter un nouveau token spécial
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # model.resize_token_embeddings(len(tokenizer))  # Adapter les embeddings

In [57]:
# Exemple de tokenisation pour vérifier
texts = ["Hello world!", "GPT-2 fine-tuning"]
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

print(encodings)

{'input_ids': tensor([[15496,   995,     0, 50256, 50256, 50256, 50256, 50256],
        [   38, 11571,    12,    17,  3734,    12, 28286,   278]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


In [58]:
# Ajouter le DataCollator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Mettez True si vous faites du Masked Language Modeling (par ex., avec BERT)
)

In [59]:
# Tokenisation
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False)  # Pas de padding ici, géré par le DataCollator

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/1127 [00:00<?, ? examples/s]

Map:   0%|          | 0/282 [00:00<?, ? examples/s]

emplacement fichier config : accelerate configuration saved at C:\Users\33760/.cache\huggingface\accelerate\default_config.yaml

In [60]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1127
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 282
    })
})

In [61]:
tokenized_datasets['train']

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1127
})

In [62]:
# Importer les modules nécessaires de la bibliothèque transformers
from transformers import TrainingArguments, Trainer

# Configurer les arguments d'entraînement pour le fine-tuning
training_args = TrainingArguments(
    output_dir="./results/trained/",
    overwrite_output_dir=False,
    #debug="underflow_overflow",
    #eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=5,
    #per_device_eval_batch_size=1,
    num_train_epochs=10,
    save_steps=5000,
    #save_total_limit=1,
    fp16=True,  # Utiliser la précision mixte si disponible
    logging_dir='./logs',
    logging_steps=20,
)

# Configurer le Trainer, une classe de Hugging Face pour gérer l'entraînement
trainer = Trainer(
    model=model,                       # Modèle à fine-tuner
    args=training_args,                # Arguments d'entraînement configurés précédemment
    train_dataset=tokenized_datasets["train"],    # Jeu de données tokenisé utilisé pour l'entraînement
    #eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Lancer le processus d'entraînement
trainer.train()

<ipython-input-62-76a76a7f6131>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
20,4.351900
40,4.065500
60,3.953500
80,3.900700
100,3.852600
120,3.793100
140,3.832200
160,3.741300
180,3.679600
200,3.699200


TrainOutput(global_step=2260, training_loss=3.175209862363022, metrics={'train_runtime': 963.8218, 'train_samples_per_second': 11.693, 'train_steps_per_second': 2.345, 'total_flos': 2743741891584000.0, 'train_loss': 3.175209862363022, 'epoch': 10.0})

In [63]:
# Sauvegarder le modèle fine-tuné dans le répertoire : ./gpt-neo-125M-fine-tuned-poetry
trainer.save_model()

In [64]:
# Sauvegarder le modèle fine-tuné
model.save_pretrained("./gpt2-poetry")
tokenizer.save_pretrained("./gpt2-poetry")

('./gpt2-poetry/tokenizer_config.json',
 './gpt2-poetry/special_tokens_map.json',
 './gpt2-poetry/vocab.json',
 './gpt2-poetry/merges.txt',
 './gpt2-poetry/added_tokens.json')

In [65]:
#generator = pipeline('text-generation', model='./gpt2-poetry', tokenizer='./gpt2-poetry')
#print(generator("Compose un poème sur le bonheur", max_length=200, temperature=0.8))


In [66]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(model_path, sequence, max_length):
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
  final_outputs = model.generate(
      ids,
      do_sample=True,
      max_length=max_length,
      pad_token_id=model.config.eos_token_id,
      top_k=50,
      top_p=0.95,
  )
  print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))


In [75]:
model2_path = "./gpt2-poetry"
sequence2 = "[Q] Nouvel amour "
max_len = 250
generate_text(model2_path, sequence2, max_len)

[Q] Nouvel amour était la grâce [A]« L'avoir je me trouve était s'apparence »
Tout moy, j'avais souvent avec son légère.
« Soutenant que ta vie et ta beauté,
Que vos cheveux, ou vos charmes, pourquoi,
Nous le décor, et nous jette le bonheur,
Que vos pensers, ne sont pas le vrai. »
L'homme, c'est là s'emporte et le bonheur,
C'est là le mien. Et laissent l'esprit était même.
Mais, à travers des pieds d'une église,
J'aime encor même plus d'une mépriser.
En lui dit était un pied fier avec vivant ;
Sans cesse à ton même aussi tout fier aucun.
En mo


In [68]:
#tokenizer.save_pretrained("./gpt-neo-125M-fine-tuned-poetry/tokenizer")

In [69]:
"""def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores['rouge1'].fmeasure
"""

"def calculate_rouge(reference, candidate):\n    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)\n    scores = scorer.score(reference, candidate)\n    return scores['rouge1'].fmeasure\n"

In [70]:
"""def generate_poem(model, tokenizer, keyword, max_length=100):
    if not keyword.strip():
        raise ValueError("Le mot-clé ne peut pas être vide. Veuillez entrer un mot valide.")

    input_ids = tokenizer.encode(keyword, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_poem"""


'def generate_poem(model, tokenizer, keyword, max_length=100):\n    if not keyword.strip():\n        raise ValueError("Le mot-clé ne peut pas être vide. Veuillez entrer un mot valide.")\n\n    input_ids = tokenizer.encode(keyword, return_tensors="pt")\n    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)\n    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)\n    return generated_poem'

In [71]:
"""def calculate_bleu(reference, candidate):
    return sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)
"""

'def calculate_bleu(reference, candidate):\n    return sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)\n'

In [72]:
"""from transformers import GPT2LMHeadModel, GPTNeoForCausalLM, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction"""
#from rouge_score import rouge_scorer

'from transformers import GPT2LMHeadModel, GPTNeoForCausalLM, GPT2Tokenizer\nfrom nltk.translate.bleu_score import sentence_bleu\nfrom nltk.translate.bleu_score import SmoothingFunction'

In [73]:
"""gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

gptneo_model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
gptneo_tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

tokenizer = AutoTokenizer.from_pretrained("./gpt-neo-125M-fine-tuned-poetry/tokenizer")
model = AutoModelForCausalLM.from_pretrained("./gpt-neo-125M-fine-tuned-poetry")
"""


'gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")\ngpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")\n\ngptneo_model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")\ngptneo_tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")\n\ntokenizer = AutoTokenizer.from_pretrained("./gpt-neo-125M-fine-tuned-poetry/tokenizer")\nmodel = AutoModelForCausalLM.from_pretrained("./gpt-neo-125M-fine-tuned-poetry")\n'

In [74]:
"""keyword = input("Entrez un mot-clé pour générer un poème : ").strip()

print(keyword)

if len(keyword) == 0:
    print("Erreur : Le mot-clé ne peut pas être vide.")
    exit()

if len(keyword) > 50:
    print("Erreur : Le mot-clé est trop long. Veuillez entrer un mot ou une courte phrase.")
    exit()

# Générer des poèmes pour chaque modèle
gpt2_poem = generate_poem(gpt2_model, gpt2_tokenizer, keyword)
#gptneo_poem = generate_poem(gptneo_model, gptneo_tokenizer, keyword)
model_poem = generate_poem(model, gptneo_tokenizer, keyword)

# Calculer les scores BLEU et ROUGE
bleu_score = calculate_bleu(gpt2_poem, model_poem)
rouge_score = calculate_rouge(gpt2_poem, model_poem)

# Afficher les poèmes générés et les scores
print("\nPoème généré par GPT-2 :\n", gpt2_poem)
#print("\nPoème généré par GPT-Neo :\n", gptneo_poem)
print("\nPoème généré par model :\n", model_poem)
print("\nScore BLEU entre les poèmes générés : ", bleu_score)
print("Score ROUGE entre les poèmes générés : ", rouge_score)"""


'keyword = input("Entrez un mot-clé pour générer un poème : ").strip()\n\nprint(keyword)\n\nif len(keyword) == 0:\n    print("Erreur : Le mot-clé ne peut pas être vide.")\n    exit()\n\nif len(keyword) > 50:\n    print("Erreur : Le mot-clé est trop long. Veuillez entrer un mot ou une courte phrase.")\n    exit()\n\n# Générer des poèmes pour chaque modèle\ngpt2_poem = generate_poem(gpt2_model, gpt2_tokenizer, keyword)\n#gptneo_poem = generate_poem(gptneo_model, gptneo_tokenizer, keyword)\nmodel_poem = generate_poem(model, gptneo_tokenizer, keyword)\n\n# Calculer les scores BLEU et ROUGE\nbleu_score = calculate_bleu(gpt2_poem, model_poem)\nrouge_score = calculate_rouge(gpt2_poem, model_poem)\n\n# Afficher les poèmes générés et les scores\nprint("\nPoème généré par GPT-2 :\n", gpt2_poem)\n#print("\nPoème généré par GPT-Neo :\n", gptneo_poem)\nprint("\nPoème généré par model :\n", model_poem)\nprint("\nScore BLEU entre les poèmes générés : ", bleu_score)\nprint("Score ROUGE entre les poème